In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
spark= SparkSession.builder.getOrCreate()

In [6]:
players= spark.read.format('csv')\
    .option("header", "True")\
    .option("inferSchema", "True")\
    .load("../data/players.csv")

In [59]:
players.show()

+--------------+--------------+----------+-----+---+----+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+
|      LastName|     FirstName|      Date|Start|Pos| Min|  G|  A| PK|PKA|  S|SoT| YK| RK|Touches|Tackles|Ints|Blocks| xG|npxG|xAG|Passes|PassesA|PrgPas|Carries|PrgCar|      Line|  C|
+--------------+--------------+----------+-----+---+----+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+
|    Aubameyang|Pierre-Emerick|2018-05-13|    1| LW|66.0|1.0|0.0|0.0|0.0|1.0|1.0|0.0|0.0|   36.0|    0.0| 0.0|   2.0|0.8| 0.8|0.0|  18.0|   31.0|   0.0|   24.0|   0.0|   Forward|  0|
|      Bellerin|        Hector|2018-05-13|    1| RB|90.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|   62.0|    1.0| 1.0|   2.0|0.0| 0.0|0.0|  42.0|   47.0|   3.0|   25.0|   2.0|  Defender|  0|
|       Holding|           Rob|2018-05-13|    1| CB|90.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|

In [8]:
players.columns

['LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C']

In [9]:
players.schema

StructType([StructField('LastName', StringType(), True), StructField('FirstName', StringType(), True), StructField('Date', StringType(), True), StructField('Start', IntegerType(), True), StructField('Pos', StringType(), True), StructField('Min', IntegerType(), True), StructField('G', IntegerType(), True), StructField('A', IntegerType(), True), StructField('PK', IntegerType(), True), StructField('PKA', IntegerType(), True), StructField('S', IntegerType(), True), StructField('SoT', IntegerType(), True), StructField('YK', IntegerType(), True), StructField('RK', IntegerType(), True), StructField('Touches', IntegerType(), True), StructField('Tackles', IntegerType(), True), StructField('Ints', IntegerType(), True), StructField('Blocks', IntegerType(), True), StructField('xG', DoubleType(), True), StructField('npxG', DoubleType(), True), StructField('xAG', DoubleType(), True), StructField('Passes', IntegerType(), True), StructField('PassesA', IntegerType(), True), StructField('PrgPas', Intege

In [10]:
players.createOrReplaceTempView("players")

In [11]:
spark.sql("""
    select concat(firstname, " ", lastname) as fullname
    from players

""").count()

2741

In [12]:
distnict_players= spark.sql("""
    select distinct concat(firstname, " ", lastname) as fullname
    from players

""")
distnict_players.show(10)

+-------------------+
|           fullname|
+-------------------+
|   Emile Smith Rowe|
|    Folarin Balogun|
|    Hector Bellerin|
|        Joe Willock|
|     William Saliba|
|       Aaron Ramsey|
|        Bukayo Saka|
|     Kieran Tierney|
|   Shkodran Mustafi|
|    Daniel Ceballos|
|Alexandre Lacazette|
|     Kieran Willian|
|      Gabriel Jesus|
|      Danny Welbeck|
| Gabriel Marquinhos|
|         Pablo Mari|
|          Ben White|
|      Ethan Nwaneri|
|     Sead Kolasinac|
|     Calum Chambers|
+-------------------+
only showing top 20 rows



In [23]:
players_Dates= spark.sql("""
    select count(distinct Date) 
    from players

""")
players_Dates.show()

+--------------------+
|count(DISTINCT Date)|
+--------------------+
|                 214|
+--------------------+



## Here we make sure that the distinct count date for matches in DimMatches Equal the distinct count Date for DimPlayers 214

In [13]:
distnict_players= distnict_players.withColumn("PlayerID", monotonically_increasing_id())

In [14]:
players= players.withColumn('fullname', concat_ws(" ", col('FirstName'),col('LastName')))
players.select("fullname").show(5, False)

+-------------------+
|fullname           |
+-------------------+
|Hector Bellerin    |
|Mohamed Elneny     |
|Rob Holding        |
|Sead Kolasinac     |
|Alexandre Lacazette|
+-------------------+
only showing top 5 rows



In [15]:
dimPlayer= players.join(distnict_players, on ='fullname', how="inner")

In [19]:
dimPlayer.write.csv('../data/Dimplayers.csv', header=True)

In [17]:
dimPlayer.select('PlayerID',
'LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA' 
).show()

+--------+---------+---------+---------+-----+---+---+---+---+---+---+
|PlayerID| LastName|FirstName|     Date|Start|Pos|Min|  G|  A| PK|PKA|
+--------+---------+---------+---------+-----+---+---+---+---+---+---+
|       2| Bellerin|   Hector|8/11/2017|    1| WB| 90|  0|  0|  0|  0|
|      49|   Elneny|  Mohamed|8/11/2017|    1| CM| 66|  0|  1|  0|  0|
|      26|  Holding|      Rob|8/11/2017|    1| CB| 66|  0|  0|  0|  0|
|      18|Kolasinac|     Sead|8/11/2017|    1| CB| 90|  0|  1|  0|  0|
|      10|Lacazette|Alexandre|8/11/2017|    1| FW| 90|  1|  0|  0|  0|
|      21|  Monreal|    Nacho|8/11/2017|    1| CB| 90|  0|  0|  0|  0|
|      53|     Ozil|    Mesut|8/11/2017|    1| AM| 90|  0|  0|  0|  0|
|      13|  Welbeck|    Danny|8/11/2017|    1| AM| 74|  1|  0|  0|  0|
|      44|    Xhaka|   Granit|8/11/2017|    1| CM| 90|  0|  2|  0|  0|
|       5|   Ramsey|    Aaron|8/11/2017|    0| DM| 24|  1|  0|  0|  0|
|       2| Bellerin|   Hector|8/19/2017|    1| WB| 90|  0|  0|  0|  0|
|     